In [1]:
import pandas as pd
import matplotlib.pyplot as mltp
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
import sklearn as sk
import seaborn as sea
import re 
import tqdm

df=pd.read_csv('Datei/blogtext.csv')

In [2]:
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niclascramer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [246]:
col=df.columns.to_list()
col

['id', 'gender', 'age', 'topic', 'sign', 'date', 'text']

In [247]:
df['gender'].value_counts()

male      345193
female    336091
Name: gender, dtype: int64

In [248]:
df['age'].value_counts()

17    80859
24    80071
23    72889
16    72708
25    67051
26    55312
27    46124
15    41767
14    27400
34    21347
33    17584
35    17462
36    14229
13    13133
37     9317
38     7545
39     5556
40     5016
45     4482
43     4230
41     3738
48     3572
42     2908
46     2733
47     2207
44     2044
Name: age, dtype: int64

In [249]:
df['topic'].value_counts()

indUnk                     251015
Student                    153903
Technology                  42055
Arts                        32449
Education                   29633
Communications-Media        20140
Internet                    16006
Non-Profit                  14700
Engineering                 11653
Law                          9040
Publishing                   7753
Science                      7269
Government                   6907
Consulting                   5862
Religion                     5235
Fashion                      4851
Marketing                    4769
Advertising                  4676
BusinessServices             4500
Banking                      4049
Chemicals                    3928
Telecommunications           3891
Accounting                   3832
Military                     3128
Museums-Libraries            3096
Sports-Recreation            3038
HumanResources               3010
RealEstate                   2870
Transportation               2326
Manufacturing 

In [250]:
df['sign'].value_counts()

Cancer         65048
Aries          64979
Taurus         62561
Libra          62363
Virgo          60399
Scorpio        57161
Pisces         54053
Leo            53811
Gemini         51985
Sagittarius    50036
Aquarius       49687
Capricorn      49201
Name: sign, dtype: int64

In [251]:
df['date'].value_counts()

02,August,2004     16544
01,August,2004     13261
03,August,2004     11851
05,August,2004      9297
04,August,2004      8661
                   ...  
10,Januar,2004         1
02,enero,2004          1
15,Mai,2004            1
08,enero,2004          1
21,October,2000        1
Name: date, Length: 2616, dtype: int64

## Data Pipline

### Preprocessing 

In [252]:
df.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...


In [253]:
df['text']=df['text'].apply(lambda x:x.lower())

In [254]:
df.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",these are the team members: drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks to yahoo!'s toolbar i can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",i had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",somehow coca-cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","if anything, korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",i surf the english news sites a l...


In [255]:
df['text'][0]

'           info has been found (+/- 100 pages, and 4.5 mb of .pdf files) now i have to wait untill our team leader has processed it and learns html.         '

In [256]:
df_pip=df.copy()
df_pip


,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",these are the team members: drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks to yahoo!'s toolbar i can ...
...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","dear susan, i could write some really ..."
681280,1713845,male,23,Student,Taurus,"01,July,2004","dear susan, 'i have the second yeast i..."
681281,1713845,male,23,Student,Taurus,"01,July,2004","dear susan, your 'boyfriend' is fuckin..."
681282,1713845,male,23,Student,Taurus,"01,July,2004","dear susan: just to clarify, i am as..."


#### removel of punktuation in the text

In [257]:
def remove_punktuation(text):
    pattern = re.compile('[<,*?>!]')
    return pattern.sub(r'', text)

In [258]:
df_pip['text']=df_pip['text'].apply(remove_punktuation)

In [259]:
#pattern =re.compile('[<,*?>]')
#pattern.search(df['text'][0])

In [260]:
def searching(text):
    pattern = re.compile('https?://S+|www.S+')
    return pattern.search(text)

In [261]:
def point_to_pip(text):
    return re.sub("([.][a-z0-9])",r'|', text)



In [262]:
df_pip['text'][0]

'           info has been found (+/- 100 pages and 4.5 mb of .pdf files) now i have to wait untill our team leader has processed it and learns html.         '

In [263]:
# pattern = re.compile('[<,*?>]')
# print(pattern.match(df_pip['text'][0]))
# txt = df_pip['text'][0]
# re.sub("([.]\s)",r'|', txt)
#df_pip['text']=df_pip['text'].apply(point_to_pip)


In [264]:
df_pip['text'][0]

'           info has been found (+/- 100 pages and 4.5 mb of .pdf files) now i have to wait untill our team leader has processed it and learns html.         '

In [265]:

df_pip[df_pip['text'].apply(searching)==True]

,id,gender,age,topic,sign,date,text


In [266]:
# from textblob import TextBlob

# def correction(text):
#     textblob = TextBlob(text)
#     return textblob.correct()

# df_pip['text']=df_pip['text'].apply(correction)

In [267]:

def spaceremove(text):
    return " ".join(text.split())
df_pip['text']=df_pip['text'].apply(spaceremove)


In [233]:
df_pip['text'][0]


'info has been found (+/- 100 pages and 4.5 mb of .pdf files) now i have to wait untill our team leader has processed it and learns html.'

In [235]:
df_pip['text'][1]

'these are the team members: drewes van der laag urllink mail ruiyu xie urllink mail bryan aaldering (me) urllink mail'

In [204]:
def delete_dupplicates(text):
    return re.sub(r'[!.]+[^a-z0-9]', '', df_pip['text'][0]) 
#df_pip['text']=df_pip['text'].apply(delete_dupplicates)

In [268]:
df_pip['text'][0]

'info has been found (+/- 100 pages and 4.5 mb of .pdf files) now i have to wait untill our team leader has processed it and learns html.'

In [269]:
df_test=df.copy()

In [270]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"U0001F600-U0001F64F"  # emoticons
                           u"U0001F300-U0001F5FF"  # symbols & pictographs
                           u"U0001F680-U0001F6FF"  # transport & map symbols
                           u"U0001F1E0-U0001F1FF"  # flags (iOS)
                           u"U00002702-U000027B0"
                           u"U000024C2-U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
#df_test['text']=df_pip['text'].apply(remove_emoji)

In [271]:
df_test['text'][0]

'info has been found (+/-  pages and . mb of .pdf files) now i have to wait untill our team leader has processed it and learns html.'

In [ ]:
#def emoji(text):
# emo = [
# [u":‑)","Happy face or smiley"],
# [u":)","Happy face or smiley"],
# [u":-]","Happy face or smiley"],
# [u":]","Happy face or smiley"],
# [u":-3","Happy face smiley"],
# [u":3","Happy face smiley"],
# [u":->","Happy face smiley"],
# [u":>","Happy face smiley"],
# [u"8-)","Happy face smiley"],
# [u":o)","Happy face smiley"],
# [u":-}","Happy face smiley"],
# [u":}","Happy face smiley"],
# [u":-)","Happy face smiley"],
# [u":c)","Happy face smiley"],
# [u":^)","Happy face smiley"],
# [u"=]","Happy face smiley"]
# ]
# for z in emo:
#     for x,y in z:
#         print(x,y)
        
            
    #     for x,y in z: 
    #         re.sub(x,y,text)
    # return text
#df_pip['text']=df_pip['text'][0].apply(emoji)

In [239]:
def tokenize(column):
    """Tokenizes a Pandas dataframe column and returns a list of tokens.

    Args:
        column: Pandas dataframe column (i.e. df['text']).

    Returns:
        tokens (list): Tokenized list, i.e. [Donald, Trump, tweets]
    """

    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]
    

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/niclascramer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# df_pip['text'] = df_pip['text'].astype(str)
# df_pip['tokenized'] = df_pip.apply(lambda x: tokenize(x['text']), axis=1)

In [272]:
df_pip['word_tokenize']=df_pip.apply(lambda row: word_tokenize(row['text']), axis=1)
df_pip['sent_tokenize']=df_pip.apply(lambda row: sent_tokenize(row['text']), axis=1)


In [279]:
df_pip.to_csv('Datei/df_tokenized.csv')

In [6]:
def stripping(liste):
    return [i.strip() for i in liste]

In [44]:
df_pip=pd.read_csv('Datei/df_tokenized.csv',converters={'sent_tokenize': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),'word_tokenize': lambda x: x[1:-1].strip('][').replace("'","").strip().split(',')})
df_pip['word_tokenize']=df_pip['word_tokenize'].apply(stripping)
df_pip['sent_tokenize']=df_pip['sent_tokenize'].apply(stripping)

In [45]:
df_pip.head(5)

,Unnamed: 0,id,gender,age,topic,sign,date,text,word_tokenize,sent_tokenize
0,0,2059027,male,15,Student,Leo,"14,May,2004",info has been found (+/- 100 pages and 4.5 mb ...,"[info, has, been, found, (, +/-, 100, pages, a...",[info has been found (+/- 100 pages and 4.5 mb...
1,1,2059027,male,15,Student,Leo,"13,May,2004",these are the team members: drewes van der laa...,"[these, are, the, team, members, :, drewes, va...",[these are the team members: drewes van der la...
2,2,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde: maak je e...,"[in, het, kader, van, kernfusie, op, aarde, :,...","[""in het kader van kernfusie op aarde: maak je..."
3,3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[testing, testing]",[testing testing]
4,4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks to yahoo's toolbar i can now 'capture' ...,"[thanks, to, yahoo, ""s"", toolbar, i, can, now,...","[""thanks to yahoos toolbar i can now capture t..."


In [46]:
df_pip['count_word']=df_pip['word_tokenize'].str.len()
df_pip['count_sent']=df_pip['sent_tokenize'].str.len()

In [47]:
df_pip.head()

,Unnamed: 0,id,gender,age,topic,sign,date,text,word_tokenize,sent_tokenize,count_word,count_sent
0,0,2059027,male,15,Student,Leo,"14,May,2004",info has been found (+/- 100 pages and 4.5 mb ...,"[info, has, been, found, (, +/-, 100, pages, a...",[info has been found (+/- 100 pages and 4.5 mb...,31,1
1,1,2059027,male,15,Student,Leo,"13,May,2004",these are the team members: drewes van der laa...,"[these, are, the, team, members, :, drewes, va...",[these are the team members: drewes van der la...,23,1
2,2,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde: maak je e...,"[in, het, kader, van, kernfusie, op, aarde, :,...","[""in het kader van kernfusie op aarde: maak je...",4783,235
3,3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[testing, testing]",[testing testing],2,1
4,4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks to yahoo's toolbar i can now 'capture' ...,"[thanks, to, yahoo, ""s"", toolbar, i, can, now,...","[""thanks to yahoos toolbar i can now capture t...",77,3


In [48]:
test=['info',
 ' has',
 ' been',
 ' found',
 ' (',
 ' +/-',
 ' 100',
 ' pages',
 ' and',
 ' 4.5',
 ' mb',
 ' of',
 ' .pdf',
 ' files',
 ' )',
 ' now',
 ' i',
 ' have',
 ' to',
 ' wait',
 ' untill',
 ' our',
 ' team',
 ' leader',
 ' has',
 ' processed',
 ' it',
 ' and',
 ' learns',
 ' html',
 ' .']
test=[i.strip() for i in test]
test

['info',
 'has',
 'been',
 'found',
 '(',
 '+/-',
 '100',
 'pages',
 'and',
 '4.5',
 'mb',
 'of',
 '.pdf',
 'files',
 ')',
 'now',
 'i',
 'have',
 'to',
 'wait',
 'untill',
 'our',
 'team',
 'leader',
 'has',
 'processed',
 'it',
 'and',
 'learns',
 'html',
 '.']

In [49]:
df_pip['word_tokenize'][0]

['info',
 'has',
 'been',
 'found',
 '(',
 '+/-',
 '100',
 'pages',
 'and',
 '4.5',
 'mb',
 'of',
 '.pdf',
 'files',
 ')',
 'now',
 'i',
 'have',
 'to',
 'wait',
 'untill',
 'our',
 'team',
 'leader',
 'has',
 'processed',
 'it',
 'and',
 'learns',
 'html',
 '.']

In [50]:
df_pip.head(5)['sent_tokenize'][2]

['"in het kader van kernfusie op aarde: maak je eigen waterstofbom how to build an h-bomb from: ascott@tartarus.uwa.edu.au (andrew scott) newsgroups: rec.humor subject: how to build an h-bomb (humorous) date: 7 feb 1994 07:41:14 gmt organization: the university of western australia original file dated 12th november 1990. seemed to be a transcript of a seven days article."',
 'poorly formatted and corrupted.',
 '"i have added the text between examine under a microscope and malleable like gold as it was missing."',
 'if anyone has the full text please distribute.',
 'i am not responsible for the accuracy of this information.',
 'converted to html by dionisio@infinet.com 11/13/98.',
 '(did a little spell-checking and some minor edits too.)',
 'stolen from urllink http://my.ohio.voyager.net/~dionisio/fun/m...own-h-bomb.html and reformatted the html.',
 'it now validates to xhtml 1.0 strict.',
 'how to build an h-bomb making and owning an h-bomb is the kind of challenge real americans seek.

In [51]:
def stopwording(liste):
    return [word for word in liste if not word in stopwords.words()]


In [52]:
from tqdm import tqdm


In [53]:
def stopwords_count_and_removal(data, col, language="english"):

    stoplist = stopwords.words(language)

    number_of_stopwords = []
    text_without_stopwords = []

    col_name_number_of_stopwords = col + "_num_of_stopwords"
    col_name_without_stopwords = col + "_without_stopwords"

    for i in tqdm(range(len(data))):
        no_of_words = 0
        without_stopwords = []
        for word in data[col][i]:
            if word in stoplist:
                no_of_words += 1
            else:
                without_stopwords.append(word)
        
        text_without_stopwords.append(without_stopwords)
        number_of_stopwords.append(no_of_words)

    data[col_name_number_of_stopwords] = number_of_stopwords
    data[col_name_without_stopwords] = text_without_stopwords


    return data

In [54]:
stopwords_count_and_removal(df_pip,'word_tokenize')

100%|██████████| 681284/681284 [04:24<00:00, 2572.58it/s]


,Unnamed: 0,id,gender,age,topic,sign,date,text,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords
0,0,2059027,male,15,Student,Leo,"14,May,2004",info has been found (+/- 100 pages and 4.5 mb ...,"[info, has, been, found, (, +/-, 100, pages, a...",[info has been found (+/- 100 pages and 4.5 mb...,31,1,12,"[info, found, (, +/-, 100, pages, 4.5, mb, .pd..."
1,1,2059027,male,15,Student,Leo,"13,May,2004",these are the team members: drewes van der laa...,"[these, are, the, team, members, :, drewes, va...",[these are the team members: drewes van der la...,23,1,4,"[team, members, :, drewes, van, der, laag, url..."
2,2,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde: maak je e...,"[in, het, kader, van, kernfusie, op, aarde, :,...","[""in het kader van kernfusie op aarde: maak je...",4783,235,1965,"[het, kader, van, kernfusie, op, aarde, :, maa..."
3,3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[testing, testing]",[testing testing],2,1,0,"[testing, testing]"
4,4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks to yahoo's toolbar i can now 'capture' ...,"[thanks, to, yahoo, ""s"", toolbar, i, can, now,...","[""thanks to yahoos toolbar i can now capture t...",77,3,29,"[thanks, yahoo, ""s"", toolbar, ""capture"", """", u..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681279,681279,1713845,male,23,Student,Taurus,"01,July,2004",dear susan i could write some really bitter di...,"[dear, susan, i, could, write, some, really, b...",[dear susan i could write some really bitter d...,45,4,17,"[dear, susan, could, write, really, bitter, di..."
681280,681280,1713845,male,23,Student,Taurus,"01,July,2004",dear susan 'i have the second yeast infection ...,"[dear, susan, """", i, have, the, second, yeast,...","[""dear susan i have the second yeast infection...",77,5,26,"[dear, susan, """", second, yeast, infection, pa..."
681281,681281,1713845,male,23,Student,Taurus,"01,July,2004",dear susan your 'boyfriend' is fucking bald go...,"[dear, susan, your, ""boyfriend"", """", is, fucki...","[""dear susan your boyfriend is fucking bald go...",13,1,4,"[dear, susan, ""boyfriend"", """", fucking, bald, ..."
681282,681282,1713845,male,23,Student,Taurus,"01,July,2004",dear susan: just to clarify i am asking you to...,"[dear, susan, :, just, to, clarify, i, am, ask...","[""dear susan: just to clarify i am asking you ...",68,2,35,"[dear, susan, :, clarify, asking, leave, house..."


In [55]:
df_pip['word_tokenize_without_stopwords'][0]

['info',
 'found',
 '(',
 '+/-',
 '100',
 'pages',
 '4.5',
 'mb',
 '.pdf',
 'files',
 ')',
 'wait',
 'untill',
 'team',
 'leader',
 'processed',
 'learns',
 'html',
 '.']

In [56]:
df_pip.head(5)

,Unnamed: 0,id,gender,age,topic,sign,date,text,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords
0,0,2059027,male,15,Student,Leo,"14,May,2004",info has been found (+/- 100 pages and 4.5 mb ...,"[info, has, been, found, (, +/-, 100, pages, a...",[info has been found (+/- 100 pages and 4.5 mb...,31,1,12,"[info, found, (, +/-, 100, pages, 4.5, mb, .pd..."
1,1,2059027,male,15,Student,Leo,"13,May,2004",these are the team members: drewes van der laa...,"[these, are, the, team, members, :, drewes, va...",[these are the team members: drewes van der la...,23,1,4,"[team, members, :, drewes, van, der, laag, url..."
2,2,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde: maak je e...,"[in, het, kader, van, kernfusie, op, aarde, :,...","[""in het kader van kernfusie op aarde: maak je...",4783,235,1965,"[het, kader, van, kernfusie, op, aarde, :, maa..."
3,3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[testing, testing]",[testing testing],2,1,0,"[testing, testing]"
4,4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks to yahoo's toolbar i can now 'capture' ...,"[thanks, to, yahoo, ""s"", toolbar, i, can, now,...","[""thanks to yahoos toolbar i can now capture t...",77,3,29,"[thanks, yahoo, ""s"", toolbar, ""capture"", """", u..."


In [57]:
# from tqdm import tqdm
# import pandas as pd
# #import swifter
# tqdm.pandas()
# df_pip['word_tokenize']=df_pip['word_tokenize'].swifter.progress_apply(stopwording)

In [58]:
a= ['info',
 'found',
 '(',
 '+/-',
 '100',
 'pages',
 '4.5',
 'mb',
 '.pdf',
 'files',
 ')',
 'wait',
 'untill',
 'team',
 'leader',
 'processed',
 'learns',
 'html',
 '.']
liste=[]
porter_stemmer = PorterStemmer()
for b in a:
    liste.append(porter_stemmer.stem(b))
liste

['info',
 'found',
 '(',
 '+/-',
 '100',
 'page',
 '4.5',
 'mb',
 '.pdf',
 'file',
 ')',
 'wait',
 'until',
 'team',
 'leader',
 'process',
 'learn',
 'html',
 '.']

In [59]:
def port(liste):
    return [porter_stemmer.stem(x) for x in liste]

In [60]:

tqdm.pandas()
df_pip['word_tokenize_without_stopwords_port']=df_pip['word_tokenize_without_stopwords'].progress_apply(port)

100%|██████████| 681284/681284 [17:45<00:00, 639.33it/s]  


deep leaning oder unseres im vergleich von hugging face oder spyre-->website beide predictions 

In [61]:
df_pip.head(5)

,Unnamed: 0,id,gender,age,topic,sign,date,text,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port
0,0,2059027,male,15,Student,Leo,"14,May,2004",info has been found (+/- 100 pages and 4.5 mb ...,"[info, has, been, found, (, +/-, 100, pages, a...",[info has been found (+/- 100 pages and 4.5 mb...,31,1,12,"[info, found, (, +/-, 100, pages, 4.5, mb, .pd...","[info, found, (, +/-, 100, page, 4.5, mb, .pdf..."
1,1,2059027,male,15,Student,Leo,"13,May,2004",these are the team members: drewes van der laa...,"[these, are, the, team, members, :, drewes, va...",[these are the team members: drewes van der la...,23,1,4,"[team, members, :, drewes, van, der, laag, url...","[team, member, :, drew, van, der, laag, urllin..."
2,2,2059027,male,15,Student,Leo,"12,May,2004",in het kader van kernfusie op aarde: maak je e...,"[in, het, kader, van, kernfusie, op, aarde, :,...","[""in het kader van kernfusie op aarde: maak je...",4783,235,1965,"[het, kader, van, kernfusie, op, aarde, :, maa...","[het, kader, van, kernfusi, op, aard, :, maak,..."
3,3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[testing, testing]",[testing testing],2,1,0,"[testing, testing]","[test, test]"
4,4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks to yahoo's toolbar i can now 'capture' ...,"[thanks, to, yahoo, ""s"", toolbar, i, can, now,...","[""thanks to yahoos toolbar i can now capture t...",77,3,29,"[thanks, yahoo, ""s"", toolbar, ""capture"", """", u...","[thank, yahoo, ""s"", toolbar, ""capture"", """", ur..."


In [62]:
df_pip.to_csv('Datei/df_tokenized_port.csv')

In [64]:
df_pip=pd.read_csv('Datei/df_tokenized.csv',converters={'sent_tokenize': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),'word_tokenize': lambda x: x[1:-1].strip('][').replace("'","").strip().split(','),
'word_tokenize_without_stopwords': lambda x: x[1:-1].strip('][').replace("'","").strip().split(',')
,'word_tokenize_without_stopwords_port': lambda x: x[1:-1].strip('][').replace("'","").strip().split(',')
})



In [ ]:
df_pip['word_tokenize']=df_pip['word_tokenize'].apply(stripping)
df_pip['sent_tokenize']=df_pip['sent_tokenize'].apply(stripping)
df_pip['word_tokenize_without_stopwords']=df_pip['word_tokenize_without_stopwords'].apply(stripping)
df_pip['word_tokenize_without_stopwords_port']=df_pip['word_tokenize_without_stopwords_port'].apply(stripping)

In [ ]:
df_pip['word_tokenize_without_stopwords_port'][0]

['info',
 'found',
 '(',
 '+/-',
 '100',
 'page',
 '4.5',
 'mb',
 '.pdf',
 'file',
 ')',
 'wait',
 'until',
 'team',
 'leader',
 'process',
 'learn',
 'html',
 '.']

# Hugging Face

In [41]:
!pip3 uninstall transformers --yes

In [67]:
!pip3 install transformers

In [69]:
!pip install flax

In [70]:
!pip3 install transformers[flax]

zsh:1: no matches found: transformers[flax]


In [50]:
!pip3 install torch torchvision torchaudio 

In [66]:
!pip install flax

In [43]:
!pip3 uninstall flax jax jaxlib numpyro --yes

Found existing installation: flax 0.6.2
Uninstalling flax-0.6.2:
  Successfully uninstalled flax-0.6.2
Found existing installation: jax 0.3.25
Uninstalling jax-0.3.25:
  Successfully uninstalled jax-0.3.25
Found existing installation: jaxlib 0.3.25
Uninstalling jaxlib-0.3.25:
  Successfully uninstalled jaxlib-0.3.25
Found existing installation: numpyro 0.10.1
Uninstalling numpyro-0.10.1:
  Successfully uninstalled numpyro-0.10.1


In [44]:
#!python3 -m pip install jax jaxlib numpyro

In [52]:
import torch as PyTorch

In [71]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

AttributeError: partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)

In [55]:
import pandas as pd
import matplotlib.pyplot as mltp
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
import sklearn as sk
import seaborn as sea
import re 
import tqdm

df=pd.read_csv('Datei/blogtext.csv')

In [56]:
df_hugging=df.copy()

In [ ]:
import os
os._exit(00)

: 

: 

In [57]:
from transformers import pipeline

AttributeError: partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)

In [58]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

AttributeError: partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)

In [59]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

NameError: name 'AutoTokenizer' is not defined

In [ ]:

model=AutoModelForSequenceClassification.from_pretrained(checkpoint)


: 

: 

In [ ]:

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
]

In [ ]:
classifier=pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)
res =classifier(raw_inputs)
print(res)

NameError: name 'model' is not defined

In [18]:
res=tokenizer(raw_inputs,padding=True,truncation=True,return_tensors="pt")
print(res)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [11]:
while torch.no_grad():
    tokens=tokenizer.tokenize(**raw_inputs)
    print(tokens)
    ids=tokenizer.convert_tokens_to_ids(tokens)
    print(ids)
    decode_str=tokenizer.decode(ids)
    print(decode_str)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']


In [12]:
ids=tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [13]:
decode_str=tokenizer.decode(ids)
print(decode_str)

i've been waiting for a huggingface course my whole life.
